# LSDL CUB, Homework 3. Constrastive Learning [10 pts]

This task is dedicated to contrastive self-supervised methods. We will focus on the SimCLR and BYOL algorithms that were discussed in class. We will conduct experiments on the [STL10](https://cs.stanford.edu/~acoates/stl10/) dataset, which is ideal for pretraining without labels, as it contains 100k unlabeled, 5k training labeled and 8k test labeled images.

To submit the task, you must conduct the experiments described in this notebook and write a report on them in PDF format. Along with the report, the code that allows you to run the experiments must be submitted. Before implementing anything, read all the experiment statements and think about how to better organize the code, do not forget to checkpoint the necessary trained models. We reserve the right to lower the grade for poorly structured code. Be sure to use the **training optimizations** from the [list](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html) in your pipelines, for example, Automatic Mixed Precision, to speed up the experiments. Also note that the report is a **mandatory part** of the assessment, without it we will not check your assignment. The report must include training curves for all the models you run. Also, make sure that your figures and graphs are readable.

## 0. Supervised baseline [0 pts]

**If not completed, the maximum for the entire task is 0 points**

We will start our study by training a supervised model from a random initial approximation. Use the labeled train for training, and the labeled test for testing. We will use ResNet-18\* as the neural network architecture. We recommend searching for hyperparameters and augmentations for training on STL-10 in articles. The author of the task got an accuracy of about 71-72%.

\**For datasets with a smaller image size than ImageNet (such as CIFAR-10/100, STL-10), it is common to use ResNet-18 with modified first layers: usually, 7x7 convolution is replaced with 3x3, and 2x2 MaxPooling is removed. We suggest you train the regular ResNet-18 in the torchvision implementation to save time.*

In [5]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stl10")

In [7]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿
import torch
import torchvision
import torchvision.transforms as transforms

from tqdm import trange

# Hyperparameters
BATCH_SIZE = 128
LEARNING_RATE = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
LR_GAMMA = 0.955
EPOCHS = 200

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Main code
model = torchvision.models.resnet18(pretrained=False).to(device)

mean_stats = [0.485, 0.456, 0.406]
std_stats = [0.229, 0.224, 0.225]
# from the previous homework
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),  # Converts image to PyTorch tensor with values in [0, 1]
    transforms.Normalize(mean=mean_stats, std=std_stats),  # Normalize the tensor,
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
])
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts image to PyTorch tensor with values in [0, 1]
    transforms.Normalize(mean=mean_stats, std=std_stats)  # Normalize the tensor
])

train_set = torchvision.datasets.STL10(root=path, split='train', download=True, transform=train_transforms)
test_set = torchvision.datasets.STL10(root=path, split='test', download=True, transform=test_transforms)    

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=LR_GAMMA)

for i in trange(EPOCHS):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        accuracy = correct / total
        print(f'Epoch {i+1}/{EPOCHS}, Test Accuracy: {accuracy:.2f}')
        
    lr_scheduler.step()


/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


  0%|          | 3964928/2640397119 [00:02<32:36, 1347211.38it/s]


KeyboardInterrupt: 

## 1. SimCLR [1.5 pts]

Implement and train the SimCLR method from [Chen et al, 2020](https://arxiv.org/pdf/2002.05709.pdf). We want you to implement it yourself, so this task does not allow you to borrow code from open sources. Use the unlabeled part of STL-10 as a training set, and use the labeled train to validate the algorithm.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿

## 2. BYOL [2.5 pts]

Similar to the previous task, implement and train the BYOL method from [Grill et al, 2020](https://arxiv.org/pdf/2006.07733.pdf). To check that the projections do not collapse (the variable $z$ from the original paper), plot the standard deviation of $z$ throughout training.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿

## 3. t-SNE [1.5 pts]

Using the t-SNE method, visualize the embeddings of images from the training and test samples that are obtained from supervised, SimCLR and BYOL models. The output of the average pooling of the model is taken as embeddings. The points corresponding to each of the 10 classes should be plotted with the same color.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿

## 4. Linear probing [1 pts]

Train a linear probe for self-supervised models, compare the quality with supervised training.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿

## 5. Fine-tuning [1.5 pts]

Finally, fine-tune the self-supervised models to STL-10 classification. If you did everything correctly, the quality should be several percent higher than that of the baseline. Similar to task 3, draw how the embeddings changed after the fine-tuning process.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿

## 6. OOD robustness [2 pts]

Now, we have 5 different models:

- Supervised
- SimCLR + linear probing
- SimCLR + fine-tuning
- BYOL + linear probing
- BYOL + fine-tuning

We will compare the models by robustness on out-of-distribution objects. As an OOD dataset, we will take the CIFAR-10 test sample, which has almost the same classes as STL-10 (9/10 classes).
The only mismatch in CIFAR-10 is the "frog" class, so drop the images of this class. Compare the trained models by OOD accuracy.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿

## Bonus. MoCo [2 pts]

As a bonus, let's look at another contrastive self-supervised model, MoCov2, from [He et al, 2019](https://arxiv.org/pdf/1911.05722), [He et al, 2020](https://arxiv.org/pdf/2003.04297.pdf). Conduct all the experiments described above with this model.

In [ ]:
# YOUR SOLUTION HERE (⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿